In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

### The Finite Difference formulation

First we make the Finite-Difference Formulation:

$$
\dfrac{\partial T}{\partial t} \approx
\begin{bmatrix}
\dfrac{-1}{2\delta t} & 0 & \dfrac{1}{2\delta t}
\end{bmatrix} \cdot
\begin{bmatrix}
T(x, \ t-\delta t) \\ T(x, \ t) \\ T(x, \ t+\delta t)
\end{bmatrix} $$
$$
\dfrac{\partial^2T}{\partial x} \approx
\begin{bmatrix}
\dfrac{1}{\delta x^2} & \dfrac{-2}{\delta x^2} & \dfrac{1}{\delta x^2}
\end{bmatrix} \cdot
\begin{bmatrix}
T(x-\delta x, \ t) \\ T(x, \ t) \\ T(x+\delta x, \ t)
\end{bmatrix} $$

Then

$$
\left[\dfrac{\partial T}{\partial t}  - \dfrac{\partial^2T}{\partial x}\right]_{\begin{matrix}x=x_i \\ t=t_j\end{matrix}} \approx 
\underbrace{\begin{bmatrix}
 \dfrac{-1}{\delta x^2}  & \dfrac{-1}{\delta t} & \dfrac{2}{\delta x^2} & \dfrac{1}{\delta t} & \dfrac{-1}{\delta x^2}
\end{bmatrix}}_{\text{stencil}}
\begin{bmatrix}
T_{j, i-1} \\ T_{j-1, i} \\ T_{j, i} \\ T_{j+1, i} \\  T_{j, i+1}
\end{bmatrix} $$

We use the notation

$$
\begin{cases}T_{j, i} = T(x_i, \ t_j)\\ T_{j, i\pm k} = T(x_i \pm k\cdot \delta x, \ t_j) \\ T_{j \pm k, i} = T(x_i, \ t_j \pm k\cdot \delta t)\end{cases} $$

There are a lot of stencils that can be used. In this example we get an error of order $\Theta(\delta x^2, \ \delta t^2)$, but there are more:

* Foward difference of $t$ (2 points) and centered on $x$ (3 points)

> \begin{align*}
\left[\dfrac{\partial T}{\partial t}\right]_{\begin{matrix}x=x_i \\ t=t_j\end{matrix}} & = \dfrac{- T_{j,i} + T_{j+1, i}}{\delta t} + \Theta (\delta t) \\
\left[\dfrac{\partial^2 T}{\partial x^2}\right]_{\begin{matrix}x=x_i \\ t=t_j\end{matrix}} & = \dfrac{T_{j,i-1}-2T_{j, i} + T_{j,i+1}}{\delta x^2} + \Theta(\delta x^2)\end{align*}

* Centered difference of $t$ (3 points) and centered on $x$ (3 points)

> \begin{align*}
\left[\dfrac{\partial T}{\partial t}\right]_{\begin{matrix}x=x_i \\ t=t_j\end{matrix}} & = \dfrac{- T_{j-1,i} + T_{j+1, i}}{2\delta t} + \Theta (\delta t^2) \\
\left[\dfrac{\partial^2 T}{\partial x^2}\right]_{\begin{matrix}x=x_i \\ t=t_j\end{matrix}} & = \dfrac{T_{j,i-1}-2T_{j, i} + T_{j,i+1}}{\delta x^2} + \Theta(\delta x^2)\end{align*}

* Centered difference of $t$ (5 points) and centered on $x$ (5 points)

> \begin{align*}
\left[\dfrac{\partial T}{\partial t}\right]_{\begin{matrix}x=x_i \\ t=t_j\end{matrix}} & = \dfrac{T_{j-2,i}-8T_{j-1, i} + 8 T_{j+1, i}-T_{j+2,i}}{12\delta t} + \Theta (\delta t^4) \\
\left[\dfrac{\partial^2 T}{\partial x^2}\right]_{\begin{matrix}x=x_i \\ t=t_j\end{matrix}} & = \dfrac{-T_{j,i-2}+16T_{j,i-1}-30T_{j,i}+16T_{j,i+1}-T_{j,i+2}}{12\delta x^2} + \Theta(\delta x^4)\end{align*}

### Exercício 1

$$
\dfrac{\partial T}{\partial t} = \dfrac{\partial^2 T}{\partial x^2} \ \ \ \ \ \forall \ (x, \ t) \ \text{on} \ \Omega
$$

With the domain $$\Omega = \left[0, \ 2\right] \times \left[0, \ \infty\right)$$

And boundary conditions

$$
T(0, \ t) = T(2, \ t) = 0 \ \ \ \ \ \forall 0 \le t
$$

$$
T(x, \ 0) = \sin \left(\frac{1}{2}\pi x\right) \ \ \ \ 0 \le x \le 2
$$

The exact solution is given by

$$
T_{exact}(x, \ t) = \exp \left(-\frac{1}{4}\pi^2 t\right) \cdot \sin \left(\frac{1}{2}\pi x\right)
$$

Now we to solve it.

Using the foward difference on $t$ and centered difference $x$ using three points we get 

$$
\dfrac{- T_{j,i} + T_{j+1, i}}{\delta t} = \dfrac{T_{j,i-1}-2T_{j, i} + T_{j,i+1}}{\delta x^2} = 0
$$

Isolating $T_{j+1, i}$ we get

$$
T_{j+1, \ i} = T_{j, \ i} + \dfrac{\delta t}{\delta x^2} \left(T_{j,i-1}-2T_{j, i} + T_{j,i+1}\right)
$$ 

Now we divide the mesh in $n_{x}$ points on $x$ domain and $n_{t}$ points on $t$ domain

In [ ]:
xmin, xmax = 0, 2
tmin, tmax = 0, 3
nx, nt = 11, 126
dx = (xmax-xmin)/(nx-1)
dt = (tmax-tmin)/(nt-1)
xmesh = np.linspace(xmin, xmax, nx)
tmesh = np.linspace(tmin, tmax, nt)

And we put the values of $T_{j}$ known at the boundary:

$$
T(0, \ t) = 0 \Rightarrow T_{j,0} = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ j = 0, \ 1, \ \cdots, \ n_t-1
$$
$$
T(2, \ t) = 0 \Rightarrow T_{j,n_x-1} = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ j = 0, \ 1, \ \cdots, \ n_t -1
$$
$$
T(x, \ 0) = \sin \dfrac{1}{2}\pi x \Rightarrow T_{0,i} = \sin \dfrac{1}{2}\pi x_i \ \ \ \ \ \ \ \ \ \ i = 0, \ 1, \ \cdots, \ n_x - 1
$$

In [ ]:
T = np.zeros((nt, nx), dtype="float64")  # Result matrix
for j, tj in enumerate(tmesh):
    T[j, 0] = 0
    T[j, nx-1] = 0
for i, xi in enumerate(xmesh):
    T[0, i] = np.sin(0.5*np.pi*xi)

In [ ]:
stencilx = np.array([1, -2, 1]) * dt/dx**2
for j in range(nt-1):
    for i in range(1, nx-1):
        T[j+1, i] = T[j, i] + stencilx @ T[j, i-1:i+2]

In [ ]:
ncurves = 5
indexs = [int(curveindex) for curveindex in np.linspace(0, nt-1, ncurves)]
plt.figure(figsize=(10, 4))
for j in indexs:
    plt.plot(xmesh, T[j, :], label=f"t = {tmesh[j]:.2}")
plt.legend()

In [ ]:
x, t = np.meshgrid(xmesh, tmesh)
maxabsT = np.max(abs(T))
fig, ax = plt.subplots(figsize=(10, 10))
dx = (xmesh[1]-xmesh[0])/2.
dy = (tmesh[1]-tmesh[0])/2.
extent = [xmesh[0]-dx, xmesh[-1]+dx, tmesh[0]-dy, tmesh[-1]+dy]
im = ax.imshow(T[::-1, :], cmap="viridis", interpolation="bicubic", aspect='auto', extent=extent)
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
div  = make_axes_locatable(ax)
cax  = div.append_axes('bottom', size='5%', pad=0.6)
cbar = plt.colorbar(im, cax=cax, orientation='horizontal')
ax.set_title(r"Numerical solution $T_{num}$")

In [ ]:
Texact = np.tensordot(np.exp(-np.pi**2 * tmesh/4), np.sin(np.pi*xmesh/2), axes=0)
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(np.abs((T-Texact)[::-1, :]), cmap="viridis", interpolation="bicubic", aspect='auto', extent=extent)
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
div  = make_axes_locatable(ax)
cax  = div.append_axes('bottom', size='5%', pad=0.6)
cbar = plt.colorbar(im, cax=cax, orientation='horizontal')
ax.set_title(r"Error of numerical solution  $|T_{num}-T_{exac}|$")

### Exercicio 2


$$
\dfrac{\partial T}{\partial t} = \dfrac{\partial^2 T}{\partial x^2} \ \ \ \ \ \forall \ (x, \ t) \ \text{on} \ \Omega
$$

With the domain $$\Omega = \left[0, \ 1\right] \times \left[0, \ \infty\right)$$

And boundary conditions

$$
T(0, \ t) = T(1, \ t) = 0 \ \ \ \ \ \forall 0 \le t
$$

$$
T(x, \ 0) = 1 \ \ \ \ 0 < x < 1
$$

The exact solution is given by

$$
T_{exact}(x, \ t) = \sum_{n=1}^{\infty} \dfrac{4}{(2n-1)\pi} \cdot \exp \left(-(2n-1)^2\pi^2 t\right) \cdot \sin \left((2n-1)\pi x\right)
$$

Now we to solve it.


In [ ]:
xmin, xmax = 0, 1
tmin, tmax = 0, 0.3
nx, nt = 51, 1497
dx = (xmax-xmin)/(nx-1)
dt = (tmax-tmin)/(nt-1)
xmesh = np.linspace(xmin, xmax, nx)
tmesh = np.linspace(tmin, tmax, nt)

T = np.zeros((nt, nx), dtype="float64")  # Result matrix
for i, xi in enumerate(xmesh):
    T[0, i] = 1
for j, tj in enumerate(tmesh):
    T[j, 0] = 0
    T[j, nx-1] = 0

stencilx = np.array([1, -2, 1]) * dt/dx**2
for j in range(nt-1):
    for i in range(1, nx-1):
        T[j+1, i] = T[j, i] + stencilx @ T[j, i-1:i+2]

In [ ]:
ncurves = 5
indexs = [int(curveindex) for curveindex in np.linspace(0, nt-1, ncurves)]
plt.figure(figsize=(10, 4))
for j in indexs:
    plt.plot(xmesh, T[j, :], label=f"t = {tmesh[j]:.2}")
plt.legend()

In [ ]:
x, t = np.meshgrid(xmesh, tmesh)
maxabsT = np.max(abs(T))
fig, ax = plt.subplots(figsize=(10, 10))
dx = (xmesh[1]-xmesh[0])/2.
dy = (tmesh[1]-tmesh[0])/2.
extent = [xmesh[0]-dx, xmesh[-1]+dx, tmesh[0]-dy, tmesh[-1]+dy]
im = ax.imshow(T[::-1, :], cmap="viridis", interpolation="bicubic", aspect='auto', extent=extent)
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
div  = make_axes_locatable(ax)
cax  = div.append_axes('bottom', size='5%', pad=0.6)
cbar = plt.colorbar(im, cax=cax, orientation='horizontal')
ax.set_title(r"Numerical solution $T_{num}$")

In [ ]:
Texact = np.zeros(T.shape)
for n in range(1, 21):
    exppart = np.exp(-(2*n-1)**2*(np.pi**2)*tmesh)
    sinpart = np.sin((2*n-1)*np.pi*xmesh)
    Texact += np.tensordot(exppart, sinpart, axes=0)/(2*n-1)
Texact *= 4/np.pi
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(np.abs((T-Texact)[::-1, :]), cmap="viridis", interpolation="bicubic", aspect='auto', extent=extent)
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
div  = make_axes_locatable(ax)
cax  = div.append_axes('bottom', size='5%', pad=0.6)
cbar = plt.colorbar(im, cax=cax, orientation='horizontal')
ax.set_title(r"Error of numerical solution  $|T_{num}-T_{exac}|$")

### Exercice 3


$$
\dfrac{\partial T}{\partial t} = \dfrac{\partial^2 T}{\partial x^2} \ \ \ \ \ \forall \ (x, \ t) \ \text{on} \ \Omega = \left[0, \ 1\right] \times \left[0, \ \infty\right)
$$

And boundary conditions

$$
T(0, \ t) = 1 \ \ \ \ \ \forall 0 \le t
$$
$$
T(1, \ t) = 0 \ \ \ \ \ \forall 0 \le t
$$

$$
T(x, \ 0) = 0 \ \ \ \ 0 < x < 1
$$

The exact solution is given by

$$
T_{exact}(x, \ t) = 1-x-\sum_{n=1}^{\infty} \dfrac{2}{n\pi} \cdot \exp \left(-n^2\pi^2 t\right) \cdot \sin \left(n\pi x\right)
$$

Now we to solve it.


In [ ]:
xmin, xmax = 0, 1
tmin, tmax = 0, 0.3
nx, nt = 51, 1497
dx = (xmax-xmin)/(nx-1)
dt = (tmax-tmin)/(nt-1)
xmesh = np.linspace(xmin, xmax, nx)
tmesh = np.linspace(tmin, tmax, nt)

T = np.zeros((nt, nx), dtype="float64")  # Result matrix
for i, xi in enumerate(xmesh):
    T[0, i] = 0
for j, tj in enumerate(tmesh):
    T[j, 0] = 1
    T[j, nx-1] = 0

stencilx = np.array([1, -2, 1]) * dt/dx**2
for j in range(nt-1):
    for i in range(1, nx-1):
        T[j+1, i] = T[j, i] + stencilx @ T[j, i-1:i+2]

In [ ]:
ncurves = 5
indexs = [int(curveindex) for curveindex in np.linspace(0, nt-1, ncurves)]
plt.figure(figsize=(10, 4))
for j in indexs:
    plt.plot(xmesh, T[j, :], label=f"t = {tmesh[j]:.2}")
plt.legend()

In [ ]:
x, t = np.meshgrid(xmesh, tmesh)
maxabsT = np.max(abs(T))
fig, ax = plt.subplots(figsize=(10, 10))
dx = (xmesh[1]-xmesh[0])/2.
dy = (tmesh[1]-tmesh[0])/2.
extent = [xmesh[0]-dx, xmesh[-1]+dx, tmesh[0]-dy, tmesh[-1]+dy]
im = ax.imshow(T[::-1, :], cmap="viridis", interpolation="bicubic", aspect='auto', extent=extent)
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
div  = make_axes_locatable(ax)
cax  = div.append_axes('bottom', size='5%', pad=0.6)
cbar = plt.colorbar(im, cax=cax, orientation='horizontal')
ax.set_title(r"Numerical solution $T_{num}$")

In [ ]:
Texact = np.zeros(T.shape)
for n in range(1, 21):
    exppart = np.exp(-n**2*(np.pi**2)*tmesh)
    sinpart = np.sin(n*np.pi*xmesh)
    Texact -= np.tensordot(exppart, sinpart, axes=0)/n
Texact *= 2/np.pi
for i, xi in enumerate(xmesh):
    Texact[:, i] += 1-xi
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(np.abs((T-Texact)[::-1, :]), cmap="viridis", interpolation="bicubic", aspect='auto', extent=extent)
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
div  = make_axes_locatable(ax)
cax  = div.append_axes('bottom', size='5%', pad=0.6)
cbar = plt.colorbar(im, cax=cax, orientation='horizontal')
ax.set_title(r"Error of numerical solution  $|T_{num}-T_{exac}|$")

### Exercice 4


$$
\dfrac{\partial T}{\partial t} = 0.01\cdot \dfrac{\partial^2 T}{\partial x^2} \ \ \ \ \ \forall \ (x, \ t) \ \text{on} \ \Omega = \left[0, \ 1\right] \times \left[0, \ \infty\right)
$$

And boundary conditions

$$
T(0, \ t) = 0 \ \ \ \ \ \forall 0 \le t
$$
$$
T(1, \ t) = 0 \ \ \ \ \ \forall 0 \le t
$$

$$
T(x, \ 0) = \begin{cases} 200x \ \ \  \ \ \  \ \ \  \ \ \ \text{if}  \ \ \ \ 0 \le x \le 0.5 \\
200(1-x) \ \ \text{if} \ \ \ \  0.5 < x \le 1 \end{cases}
$$

The exact solution is given by

$$
T_{exact}(x, \ t) = \dfrac{800}{\pi^2}\sum_{n=1}^{\infty} \dfrac{(-1)^{n}}{(2n+1)^2} \cdot \exp \left(-(2n+1)^2\pi^2 \cdot 0.01 \cdot t\right) \cdot \sin \left((2n+1)\pi x\right)
$$

Now we to solve it.


In [ ]:
xmin, xmax = 0, 1
tmin, tmax = 0, 3
nx, nt = 51, 10000
dx = (xmax-xmin)/(nx-1)
dt = (tmax-tmin)/(nt-1)
xmesh = np.linspace(xmin, xmax, nx)
tmesh = np.linspace(tmin, tmax, nt)

T = np.zeros((nt, nx), dtype="float64")  # Result matrix
for i, xi in enumerate(xmesh):
    if xi > 0.5:
        T[0, i] = 200*(1-xi)
    else:
        T[0, i] = 200*xi
for j, tj in enumerate(tmesh):
    T[j, 0] = 0
    T[j, nx-1] = 0

stencilx = 0.01*np.array([1, -2, 1]) * dt/dx**2
for j in range(nt-1):
    for i in range(1, nx-1):
        T[j+1, i] = T[j, i] + stencilx @ T[j, i-1:i+2]

In [ ]:
ncurves = 5
indexs = [int(curveindex) for curveindex in np.linspace(0, nt-1, ncurves)]
plt.figure(figsize=(10, 4))
for j in indexs:
    plt.plot(xmesh, T[j, :], label=f"t = {tmesh[j]:.2}")
plt.legend()

In [ ]:
x, t = np.meshgrid(xmesh, tmesh)
maxabsT = np.max(abs(T))
fig, ax = plt.subplots(figsize=(10, 10))
dx = (xmesh[1]-xmesh[0])/2.
dy = (tmesh[1]-tmesh[0])/2.
extent = [xmesh[0]-dx, xmesh[-1]+dx, tmesh[0]-dy, tmesh[-1]+dy]
im = ax.imshow(T[::-1, :], cmap="viridis", interpolation="bicubic", aspect='auto', extent=extent)
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
div  = make_axes_locatable(ax)
cax  = div.append_axes('bottom', size='5%', pad=0.6)
cbar = plt.colorbar(im, cax=cax, orientation='horizontal')
ax.set_title(r"Numerical solution $T_{num}$")

In [ ]:
Texact = np.zeros(T.shape)
for n in range(0, 21):
    exppart = np.exp(-(2*n+1)**2*(np.pi**2)*0.01*tmesh)
    sinpart = np.sin((2*n+1)*np.pi*xmesh)
    Texact += (-1)**n * np.tensordot(exppart, sinpart, axes=0)/((2*n+1)**2)
Texact *= 800/(np.pi**2)
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(np.abs((T-Texact)[::-1, :]), cmap="viridis", interpolation="bicubic", aspect='auto', extent=extent)
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
div  = make_axes_locatable(ax)
cax  = div.append_axes('bottom', size='5%', pad=0.6)
cbar = plt.colorbar(im, cax=cax, orientation='horizontal')
ax.set_title(r"Error of numerical solution  $|T_{num}-T_{exac}|$")

### Exercice 5


$$
\dfrac{\partial T}{\partial t} = 0.01\cdot \dfrac{\partial^2 T}{\partial x^2} \ \ \ \ \ \forall \ (x, \ t) \ \text{on} \ \Omega = \left[0, \ 0.5\right] \times \left[0, \ \infty\right)
$$

And boundary conditions

$$
T(0, \ t) = 0 \ \ \ \ \ \forall 0 \le t
$$
$$
\dfrac{\partial T}{\partial x}(0.5, \ t) = 0 \ \ \ \ \ \forall 0 \le t
$$
$$
T(x, \ 0) = 200x \ \ \ \ \ \forall  \ 0 \le x \le 0.5
$$

The exact solution is given by

$$
T_{exact}(x, \ t) = \dfrac{800}{\pi^2}\sum_{n=1}^{\infty} \dfrac{(-1)^{n}}{(2n+1)^2} \cdot \exp \left(-(2n+1)^2\pi^2 \cdot 0.01 \cdot t\right) \cdot \sin \left((2n+1)\pi x\right)
$$

Now we to solve it.


In [ ]:
xmin, xmax = 0, 0.5
tmin, tmax = 0, 3
nx, nt = 51, 10000
dx = (xmax-xmin)/(nx-1)
dt = (tmax-tmin)/(nt-1)
xmesh = np.linspace(xmin, xmax, nx)
tmesh = np.linspace(tmin, tmax, nt)

T = np.zeros((nt, nx), dtype="float64")  # Result matrix
for j, tj in enumerate(tmesh):
    T[j, 0] = 0
for i, xi in enumerate(xmesh):
    T[0, i] = 200*xi


stencilx = 0.01*np.array([1, -2, 1]) * dt/dx**2
for j in range(nt-1):
    for i in range(1, nx-1):
        T[j+1, i] = T[j, i] + stencilx @ T[j, i-1:i+2]
    T[j+1, nx-1] = T[j, nx-1]

In [ ]:
ncurves = 5
indexs = [int(curveindex) for curveindex in np.linspace(0, nt-1, ncurves)]
plt.figure(figsize=(10, 4))
for j in indexs:
    plt.plot(xmesh, T[j, :], label=f"t = {tmesh[j]:.2}")
plt.legend()

In [ ]:
x, t = np.meshgrid(xmesh, tmesh)
maxabsT = np.max(abs(T))
fig, ax = plt.subplots(figsize=(10, 10))
dx = (xmesh[1]-xmesh[0])/2.
dy = (tmesh[1]-tmesh[0])/2.
extent = [xmesh[0]-dx, xmesh[-1]+dx, tmesh[0]-dy, tmesh[-1]+dy]
im = ax.imshow(T[::-1, :], cmap="viridis", interpolation="bicubic", aspect='auto', extent=extent)
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
div  = make_axes_locatable(ax)
cax  = div.append_axes('bottom', size='5%', pad=0.6)
cbar = plt.colorbar(im, cax=cax, orientation='horizontal')
ax.set_title(r"Numerical solution $T_{num}$")

In [ ]:
Texact = np.zeros(T.shape)
for n in range(0, 21):
    exppart = np.exp(-(2*n+1)**2*(np.pi**2)*0.01*tmesh)
    sinpart = np.sin((2*n+1)*np.pi*xmesh)
    Texact += (-1)**n * np.tensordot(exppart, sinpart, axes=0)/((2*n+1)**2)
Texact *= 800/(np.pi**2)
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(np.abs((T-Texact)[::-1, :]), cmap="viridis", interpolation="bicubic", aspect='auto', extent=extent)
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
div  = make_axes_locatable(ax)
cax  = div.append_axes('bottom', size='5%', pad=0.6)
cbar = plt.colorbar(im, cax=cax, orientation='horizontal')
ax.set_title(r"Error of numerical solution  $|T_{num}-T_{exac}|$")

### Exercicios 6

$$
\dfrac{\partial T}{\partial t} = \dfrac{\partial^2 T}{\partial x^2}+\dfrac{\partial^2 T}{\partial y^2}\ \ \ \ \ \ \forall \left(x, \ y,\ t\right)\text{on} \ \Omega = \left[0,\ 1\right]\times\left[0,\ 1\right]\times\left[0,\ \infty\right)
$$

And boundary conditions

\begin{align*}
T(x, \ 0, \ t) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(x, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right) \\
T(x, \ 1, \ t) & = \sin \left(\pi x\right) \ \ \ \ \ \forall \left(x, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right)  \\
T(0, \ y, \ t) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(y, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right) \\
T(1, \ y, \ t) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(y, \ t\right) \in \left[0, \ 1\right]\times\left[0,\ \infty\right)\\
T(x, \ y, \ 0) & = 0 \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \forall \left(x, \ y\right) \in \left[0, \ 1\right]\times\left[0, \ 1\right]
\end{align*}

The exact stationary solution when $t\to \infty$ is given by

$$
T_{exact}(x, \ y,\ t\to \infty) = \dfrac{\sinh (\pi y) \sin (\pi x)}{\sinh \pi}
$$

Now we to solve it.

In [ ]:
xmin, xmax = 0, 1
ymin, ymax = 0, 1
tmin, tmax = 0, 3
nx, ny, nt = 10, 10, 1000
dx = (xmax-xmin)/(nx-1)
dy = (ymax-ymin)/(ny-1)
dt = (tmax-tmin)/(nt-1)
xmesh = np.linspace(xmin, xmax, nx)
ymesh = np.linspace(ymin, ymax, ny)
tmesh = np.linspace(tmin, tmax, nt)

T = np.zeros((nt, ny, nx), dtype="float64")  # Result matrix
for k, tk in enumerate(tmesh):
    T[k, ny-1, :] = np.sin(np.pi*xmesh)

stencilx = np.array([1, -2, 1])*dt/(dx**2)
stencily = np.array([1, -2, 1])*dt/(dy**2)
for k in range(nt-1):
    for i in range(1, nx-1):
        for j in range(1, ny-1):
            T[k+1, j, i] = T[k, j, i] + stencilx @ T[k, j, i-1:i+2] + stencily @ T[k, j-1:j+2, i]

In [ ]:
x, y = np.meshgrid(xmesh, ymesh)
fig, ax = plt.subplots(figsize=(10, 10))
dx = (xmesh[1]-xmesh[0])/2.
dy = (ymesh[1]-ymesh[0])/2.
extent = [xmesh[0]-dx, xmesh[-1]+dx, ymesh[0]-dy, ymesh[-1]+dy]
im = ax.imshow(T[-1], cmap="viridis", interpolation="bicubic", aspect='auto', extent=extent)
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Time $t$")
div  = make_axes_locatable(ax)
cax  = div.append_axes('bottom', size='5%', pad=0.6)
cbar = plt.colorbar(im, cax=cax, orientation='horizontal')
ax.set_title(r"Numerical solution $T_{num}$ at final time")

In [ ]:
Tinfy = np.tensordot(np.sinh(np.pi*ymesh), np.sin(np.pi*xmesh),axes=0)/np.sinh(np.pi)
diff = np.abs(Tinfy-T[-1])
fig, ax = plt.subplots(figsize=(10, 10))
extent = [xmesh[0]-dx, xmesh[-1]+dx, tmesh[0]-dy, tmesh[-1]+dy]
im = ax.imshow(diff, cmap="viridis", interpolation="bicubic", aspect='auto', extent=extent)
ax.set_xlabel("Space in $x$")
ax.set_ylabel("Space in $y$")
div  = make_axes_locatable(ax)
cax  = div.append_axes('bottom', size='5%', pad=0.6)
cbar = plt.colorbar(im, cax=cax, orientation='horizontal')
ax.set_title(r"Error of numerical solution  $|T_{num}-T_{exac}|$")